# Data Cleaning and Preparation

Data preparation often takes about 80% of time.

In [ ]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [ ]:
import numpy as np
import pandas as pd
PREVIOUS_MAX_ROWS = pd.options.display.max_rows
pd.options.display.max_rows = 20
np.random.seed(12345)
import matplotlib.pyplot as plt
plt.rc('figure', figsize=(10, 6))
np.set_printoptions(precision=4, suppress=True)

## Handling Missing Data

* `dropna`
* `fillna`
* `isnull`
* `notnull`

In [ ]:
string_data = pd.Series(['aardvark', 'artichoke', np.nan, 'avocado'])
string_data
string_data.isnull()

Do you remember what  NaN stands for?
In Pandas, it is used to represent missing data.

In [ ]:
string_data[0] = None
string_data.isnull()

In [ ]:
string_data

Note that both `NaN` and `None` are evalutated by `isnull()` to `True`.
The `None` value can be read as `Not Available` (NA).

In summary, missing data can be referred to using the following keywords:
* NaN (Not a Number)
* None (used to signify the null value)

Generally, in data science, these additional terms are used:
* NA (Not Available)
* null


### Filtering Out Missing Data

#### On Numpy Series
null values are omitted

In [ ]:
from numpy import nan as NA
data = pd.Series([1, NA, 3.5, NA, 7])
data
data.dropna()

Why is the following statement equivallent?

In [ ]:
data
data.notnull()

data[data.notnull()]

#### On Pandas Dataframes

`df.dropna`:  if a row contains *any* missing value, it is by default dropped.

In [ ]:
data = pd.DataFrame([[1., 6.5, 3.], [1., NA, NA],
                     [NA, NA, NA], [NA, 6.5, 3.]])
cleaned = data.dropna()
data


On  many datasets, this would filter out too many rows:

In [ ]:
cleaned

We can modify the conditions for dropping a row by specifying `how`.
* any : if any NA values are present, drop that label
* all : if all values are NA, drop that label

By default, row axis  is used.

In [ ]:
data.dropna(how='all')

In [ ]:
data.dropna?

By specifying the axis argument, we can remove columns with missing values.

In [ ]:
data[4] = NA
data

data.dropna(axis=1, how='all')

In [ ]:
df = pd.DataFrame(np.random.randn(7, 3))
df.iloc[:4, 1] = NA
df.iloc[:2, 2] = NA
df
df.dropna()
df.dropna(thresh=2)

### Filling In Missing Data

Many analytical methods and machine learning algorithms will not accept datasets with missing data.

An alternative to removing rows/columns with missing data is replacing the missing observations with some specific value.

#### replace all missing observations with the same value

In [ ]:
df
df.fillna(0)

#### specify replacement value for each column separately

In [ ]:
df.fillna({1: 0.5, 2: 0})

#### specify in place replacement

In [ ]:
_ = df.fillna(0, inplace=True)
df

#### specify filling method and limit

method : {'backfill', 'bfill', 'pad', 'ffill', None}, default None
    
    Method to use for filling holes in reindexed Series  
    pad / ffill: propagate last valid observation forward to next valid
    backfill / bfill: use NEXT valid observation to fill gap


In [ ]:
# create data frame with random values (normally distributed)
df = pd.DataFrame(np.random.randn(6, 3))
# set some elements of the data frame as missing
df.iloc[2:, 1] = NA
df.iloc[4:, 2] = NA
df
df.fillna(method='ffill')

#### specify filling limit
limit : int, default None

    If method is specified, this is the maximum number of consecutive
    NaN values to forward/backward fill. In other words, if there is
    a gap with more than this number of consecutive NaNs, it will only
    be partially filled. If method is not specified, this is the
    maximum number of entries along the entire axis where NaNs will be
    filled.

In [ ]:
df
df.fillna(method='ffill', limit=2)

In [ ]:
?df.fillna

In [ ]:
data = pd.Series([1., NA, 3.5, NA, 7])
data.fillna(data.mean())

## Data Transformation

### Removing Duplicates

A  *duplicate* typically means a row that has the same values in all columns except the index as some other row in the same data frame.

In [ ]:
# create df containing duplicate rows
data = pd.DataFrame({'k1': ['one', 'two'] * 3 + ['two'],
                     'k2': [1, 1, 2, 3, 3, 4, 4]})
data

#### Determining whether the row is a duplicate

In [ ]:
data.duplicated()

#### removing duplicates

In [ ]:
data.drop_duplicates()

#### determine duplicate status only based on selected columns

In [ ]:
data
data.duplicated()
data.drop_duplicates(['k1'])

#### changing which observation will be kept

In [ ]:
data.drop_duplicates(['k1'], keep='last')

### Transforming Data Using a Function or Mapping

Example problem: data table with meat types and weight

In [ ]:
data = pd.DataFrame({'food': ['bacon', 'pulled pork', 'bacon',
                              'Pastrami', 'corned beef', 'Bacon',
                              'pastrami', 'honey ham', 'nova lox'],
                     'ounces': [4, 3, 12, 6, 7.5, 8, 3, 5, 6]})
data

Define a mapping specifying which type of animal the meat came from.

In [ ]:
meat_to_animal = {
  'bacon': 'pig',
  'pulled pork': 'pig',
  'pastrami': 'cow',
  'corned beef': 'cow',
  'honey ham': 'pig',
  'nova lox': 'salmon'
}

What data type is `meat_to_animal`?

#### unify capitalization
Notice that `meat_to_animal` and `data` use different capitalization.

In [ ]:
lowercased = data['food'].str.lower()
lowercased

#### Performing the mapping using map() function on Series
Map values of Series using input correspondence (which can be
a dict, Series, or function)

In [ ]:
# meat_to_animal is a dictionary with keys such as "bacon" and value such as "pig"
# lowercased is also a food such as "bacon"
data['animal'] = lowercased.map(meat_to_animal)
#the result is a new column in data with elements corresponding to values in meat_to_animal
data

#### An alternative solution using a lambda expression
The original solution:

    `lowercased = data['food'].str.lower()`
    `data['animal'] = lowercased.map(meat_to_animal)`
can be replaced by one liner:

In [ ]:
data['animal'] = data['food'].map(lambda x: meat_to_animal[x.lower()])
data['animal']

Note that 
* data['food'].map(lambda x: meat_to_animal[x.lower()])
* lowercased.map(meat_to_animal)

is the same function called on Series, however, different types of arguments are passed:

* data['food'].map():   we passed a function
* lowercased.map(): we passed a dictionary


### Replacing Values

A frequent scenario when we want to replace a missing value is the handling of outliers.
While outlier is not a missing value, it can be treated as so.

In [ ]:
# value -999 will indicate an outlier
data = pd.Series([1., -999., 2., -999., -1000., 3.])
data

#### Replacing a specific value with a specific value

In [ ]:
data.replace(-999, np.nan)

#### Replacing one value from a list with a specific value

In [ ]:
data.replace([-999, -1000], np.nan)

#### Replacing list of values with a list of values (two lists solution)

In [ ]:
data.replace([-999, -1000], [np.nan, 0])

#### Replacing list of values with a list of values (one dict solution)

In [ ]:
data.replace({-999: np.nan, -1000: 0})

### Renaming Axis Indexes (advanced)

In [ ]:
data = pd.DataFrame(np.arange(12).reshape((3, 4)),
                    index=['Ohio', 'Colorado', 'New York'],
                    columns=['one', 'two', 'three', 'four'])

In [ ]:
transform = lambda x: x[:4].upper()
data.index.map(transform)

In [ ]:
data.index = data.index.map(transform)
data

#### create a modified version of df without changing the original

In [ ]:
data.rename(index=str.title, columns=str.upper)

In [ ]:
data.rename(index={'OHIO': 'INDIANA'},
            columns={'three': 'peekaboo'})

In [ ]:
data.rename(index={'OHIO': 'INDIANA'}, inplace=True)
data

### Discretization and Binning

`pd.cut(x, bins, right=True, labels=None, retbins=False, precision=3, include_lowest=False)`

    x : array-like
        Input array to be binned. It has to be 1-dimensional.
    bins : int, sequence of scalars, or IntervalIndex
        If `bins` is an int, it defines the number of equal-width bins in the
        range of `x`. However, in this case, the range of `x` is extended
        by .1% on each side to include the min or max values of `x`. If
        `bins` is a sequence it defines the bin edges allowing for
        non-uniform bin width. No extension of the range of `x` is done in
        this case.
    right : bool, optional
        Indicates whether the bins include the rightmost edge or not. If
        right == True (the default), then the bins [1,2,3,4] indicate
        (1,2], (2,3], (3,4].
    labels : array or boolean, default None
        Used as labels for the resulting bins. Must be of the same length as
        the resulting bins. If False, return only integer indicators of the
        bins.
    retbins : bool, optional
        Whether to return the bins or not. Can be useful if bins is given
        as a scalar.
    precision : int, optional
        The precision at which to store and display the bins labels
    include_lowest : bool, optional
        Whether the first interval should be left-inclusive or not.


In [ ]:
ages = [22, 22, 25, 27, 21, 23, 37, 31, 61, 45, 41, 32]

Divide data to bins
* (18, 25]
* ...
* (60, 100]

Where () means "open"=exclusive, and [] means "closed" = inclusive

#### Binning with manual definition of boundaries

In [ ]:
bins = [18, 25, 35, 60, 100]
cats = pd.cut(ages, bins)
ages
cats

#### Categorical.codes: which value is assigned to which category (bin)
Applicable when cut function returns the Categorical data type.

In [ ]:
ages
cats.codes

#### Which bins were produced by the discretization

In [ ]:
cats.categories # list of categories

#### What is the distribution of values

In [ ]:
pd.value_counts(cats)

Note that the function counts all values, not just unique values.

In(18, 25] there are four distinct values in `ages` and five values in total: [21, 22, 22, 23, 25]

`pd.value_counts(cats)` returns 5

#### Changing which side of the produced intervals is closed

In [ ]:
pd.cut(ages, [18, 26, 36, 61, 100], right=False)

#### Specifying labels

In [ ]:
group_names = ['Youth', 'YoungAdult', 'MiddleAged', 'Senior']
pd.cut(ages, bins, labels=group_names)

#### Equidistant binning
Created bins have the same lenght.
This option is activated when bin count is passed instead of bin boundaries.

In [ ]:
data = np.random.rand(20)
pd.cut(data, 4)

#### Rounding bin boundaries

Rounding precision is specified using the `precision` argument.

In [ ]:
pd.cut(data, 4,precision=2)

#### Equal size binning

* Desirable if you want the bins to cover the same number of data points
* Called quantile cut `qcut`

In [ ]:
data = np.random.randn(1000)
# the second argument is the number of bins produced
cats = pd.qcut(data,4)
cats
pd.value_counts(cats)

In [ ]:
pd.qcut(data, [0, 0.1, 0.5, 0.9, 1.])

### Detecting and Filtering Outliers

Wikipedia has an [excellent article](https://en.wikipedia.org/wiki/Outlier) on outliers:
"In statistics, an outlier is an observation point that is distant from other observations. An outlier may be due to variability in the measurement or it may indicate experimental error; the latter are sometimes excluded from the data set. An outlier can cause serious problems in statistical analyses."

There is no universally applicable test for outlier detection!


In [ ]:
data = pd.DataFrame(np.random.randn(1000, 4))
data
data.describe()

#### Are there any values outside a prespecified range?
Assume that for the third column, anything outside the [-3;3] range is considered as an oulier.

In [ ]:
col = data[2]
col
col[np.abs(col) > 3]

In [ ]:
# any: Return whether any element is True over requested axis
data[(np.abs(data) > 3).any(1)]

In [ ]:
how this works:

In [ ]:
np.abs(data)
np.abs(data) > 3
# this will return true iff in given  row there is at least one value outside [-3;3]
(np.abs(data) > 3).any(1)

#### cap outlying values

If a value is outside the specified interval, it will be replaced by the boundary. The sign of the value is kept.

In [ ]:
# np.sign(data) outputs 1 when value is positive and -1 when it is negative
data[np.abs(data) > 3] = np.sign(data) * 3
data.describe()

### Permutation and Random Sampling

Permutation is random reodering.

First, let us define a data frame in which we will want to reorder the rows.

In [ ]:
# this is same as np.arange(20)
df = pd.DataFrame(np.arange(5 * 4).reshape((5, 4)))
df

Next, we define a new ordering. Since we have five rows in the dataframe, we want a random permutation of length 5.

In [ ]:
sampler = np.random.permutation(5)
sampler

In [ ]:
Now, we reorder the data frame using the permutation.

In [ ]:
df
# take: Take elements from an array along an axis.
df.take(sampler)

#### Create subset without replacement

In [ ]:
df.sample(n=3)

The limitation is that `sample` cannot take a larger sample than population. 

In [ ]:
# will throw an error
df.sample(n=10)

##### Sample with replacement 

pass `replace=True` argument to the sample function

In [ ]:
choices = pd.Series([5, 7, -1, 6, 4])
draws = choices.sample(n=10, replace=True)
draws

### Computing Indicator/Dummy Variables

Some machine learning or classification algorithms require that explanatory or target variables are numeric.

| id | var1 |   |   |   |
|----|------|---|---|---|
| 1  | a    |   |   |   |
| 2  | b    |   |   |   |
| 3  | c    |   |   |   |
=>

| id | var1=a | var1=b | var1=c |   |
|----|--------|--------|--------|---|
| 1  | 1      | 0      | 0      |   |
| 2  | 0      | 1      | 1      |   |
| 3  | 0      | 0      | 1      |   |

In [ ]:
df = pd.DataFrame({'key': ['b', 'b', 'a', 'c', 'a', 'b'],
                   'data1': range(6)})
df
# this will dummify only the selected column
pd.get_dummies(df['key'])

Note that, by default, pandas will not place the name of the original variable to the column heading. If you want to preserve it, you can use the `prefix` argument.

In [ ]:
dummies = pd.get_dummies(df['key'], prefix='key')
dummies

In the following, we will append the newly created dataframe to the `data1` column from the original data frame.

In [ ]:
df_with_dummy = df[['data1']].join(dummies)
df_with_dummy

#### row belongs to multiple categories (advanced)

In [ ]:
mnames = ['movie_id', 'title', 'genres']
movies = pd.read_table('datasets/movielens/movies.dat', sep='::',
                       header=None, names=mnames)
movies[:10]

##### First step: Identify unique genres:

In [ ]:
all_genres = []
for x in movies.genres:
    all_genres.extend(x.split('|'))
genres = pd.unique(all_genres)

In [ ]:
genres

##### Second step: initialize matrix to all zeros

In [ ]:
# the dimensions: same number of rows as original matrix, for each distinct genre one column
zero_matrix = np.zeros((len(movies), len(genres)))
# create a data frame from the numpy array, name columns
dummies = pd.DataFrame(zero_matrix, columns=genres)
dummies

##### Third step: identify which positions to set to 1

In [ ]:
The principle

In [ ]:
# select  genre  of the second film
gen = movies.genres[1] 
#identify the composing values 
gen.split('|')
# returns positions of columns with name matching the passed list of genres
dummies.columns.get_indexer(gen.split('|')) 

In [ ]:
Putting it together

In [ ]:
# for each row i in the original matrix, get the string with genres
for i, gen in enumerate(movies.genres):
    #get the columns in the dummies matrix that should be set to 1
    indices = dummies.columns.get_indexer(gen.split('|'))
    # and set them to 1 for the right row
    dummies.iloc[i, indices] = 1

In [ ]:
# append the created dummies matrix to the original matrix
# specify a prefix for all newly added columns
movies_windic = movies.join(dummies.add_prefix('Genre_'))
# display the first row
movies_windic.iloc[1]

#### Combine dummification with discretization

In [ ]:
np.random.seed(12345)
values = np.random.rand(10)
values
bins = [0, 0.2, 0.4, 0.6, 0.8, 1]
# dummify created bins
pd.get_dummies(pd.cut(values, bins), prefix="val")

## String Manipulation

### String Object Methods

In [ ]:
val = 'a,b,  guido'
val.split(',')

values = val.split(',')

#### trim white space

In [ ]:
pieces = [x.strip() for x in val.split(',')]
pieces

#### concatenation on strings: addition

In [ ]:
first, second, third = pieces
first + '::' + second + '::' + third

#### concatenation on strings: join method
This is faster than the + operator

In [ ]:
pieces
'::'.join(pieces)

#### searching in substrings

* `in` returns a boolean, not position
* `index` returns position, exception if string is not found
* `find` returns position, -1 if string is not found

In [ ]:
'guido' in val
val.index(',')
val.find(':')

In [ ]:
# will throw an error
val.index(':')

#### counting occurrences in strings

In [ ]:
val.count(',')

#### substitution in strings

In [ ]:
val
val.replace(',', '::')
val.replace(',', '')

### Regular Expressions

Subtopics
* splitting
* pattern matching
* substitution


### Splitting
#### split with variable number of whitespace chars

In [ ]:
import re
text = "foo    bar\t baz  \tqux"
re.split('\s+', text)

#### speeding up matching with precompilation of regex
This is particularly useful if the expression is reused.

In [ ]:
regex = re.compile('\s+')
regex.split(text)

### Pattern matching
#### findall: get a list of all matches in the input string

In [ ]:
text
regex.findall(text)

In [ ]:
text = """Dave dave@google.com
Steve steve@gmail.com
Rob rob@gmail.com
Ryan ryan@yahoo.com
"""
pattern = r'[A-Z0-9._%+-]+@[A-Z0-9.-]+\.[A-Z]{2,4}'

# re.IGNORECASE makes the regex case-insensitive
regex = re.compile(pattern, flags=re.IGNORECASE)

In [ ]:
regex.findall(text)

#### search: return only the first match

In [ ]:
m = regex.search(text)
m
text[m.start():m.end()]

#### match: only match at the beginning of the string

In [ ]:
print(regex.match(text))

### Substitution

* return a new string with occurrences of the regex pattern replaced by the specified string

In [ ]:
print(regex.sub('REDACTED', text))

### Grouping
Using groups in regular expression allows to extract parts of the string matched by the expression

Example
* find email addresses
* extract user name, domain name, domain suffix

In [ ]:
# parenthesis () in the regex  are used to delimit the groups
pattern = r'([A-Z0-9._%+-]+)@([A-Z0-9.-]+)\.([A-Z]{2,4})'
regex = re.compile(pattern, flags=re.IGNORECASE)

In [ ]:
m = regex.match('wesm@bright.net')
m.groups()

In [ ]:
regex.findall(text)

#### Combining grouping with substitution

In Python, a group is matched in substition using `\n` syntax, where `n` is the number of the group (starting with 1)

In [ ]:
# remember that regex is the compiled regular expression
# first argument specifies what substitution should be performed on each of the matched groups
# second argument is the passed text
text
print(regex.sub(r'Username: \1, Domain: \2, Suffix: \3', text))

##### How this works?
Consider input string: **Dave dave@google.com**
* Dave = not matched by the regex
* dave => Username: dave
* @ => matched by the regex, but not included in any of the groups, therefore discarded
* google => Domain: google
* . => matched by the regex, but not included in any of the groups, therefore discarded
* com => Suffix: com

### Vectorized String Functions in pandas

When data contain missing values, some string functions will result in error.

In [ ]:
# this will throw an error
wesEmail = None
"@" in wesEmail

Solution is to used vectorized operations on Series and Data Frame.

In [ ]:
data = {'Dave': 'dave@google.com', 'Steve': 'steve@gmail.com',
        'Rob': 'rob@gmail.com', 'Wes': np.nan}
data = pd.Series(data)
data
data.isnull()

In the following data is a Series

#### vectorized string operations

In [ ]:
type(data)
data.str.contains('gmail')

#### vectorized regex matching

In [ ]:
pattern
data.str.findall(pattern, flags=re.IGNORECASE)

In [ ]:
matches = data.str.match(pattern, flags=re.IGNORECASE)
matches

#### vectorized element retrieval

In [ ]:
matches.str.get(1) 
matches.str[0]

#### vectorized slicing

In [ ]:
data.str[:5]

In [ ]:
pd.options.display.max_rows = PREVIOUS_MAX_ROWS

## Reading

Chapter 7 in Python Data Analysis.

## Excercise

Cleaning data for analysis from Cleaning Data in Python


## Home assignment

Complete Cleaning Data from pandas Puzzles

## License and acknowledgment
The code in this document contains code from supplementary material for Python for Data Analysis (Wes McKinney). 
This notebook is released under the MIT license. 

Code examples from "Python for Data Analysis", 2nd Edition

The MIT License (MIT)

Copyright (c) 2017 Wes McKinney

Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all
copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
SOFTWARE.